## Cruzamento dos Dados (Enunciado Questões + Microdados)

* **Input**: Local do CSV de Questões Extraídas + Local do CSV de Microdados

* **Output**: CSV com as colunas -> *numero_questao*, *enunciado*, *alternativas*, *nu_param_B* e *gabarito*

In [61]:
import pandas as pd

In [120]:
# Definição de Variáveis de Input e Output
year = "2015"

question_path = f"../data/extracted_questions/enem_{year}.csv"
microdados_path = f"../data/microdados/microdados_{year}.csv"
output_path = f"../data/merged_data/enem_{year}.csv"

In [121]:
# Ler o CSV extraído do PDF (resultado de Text_Extraction.ipynb)
df_questions = pd.read_csv(question_path, encoding="utf-8", quotechar='"')

# Ler o CSV de microdados
df_microdados = pd.read_csv(microdados_path, delimiter=";", encoding="latin1")

In [122]:
co_prova = {
    "2015": 275,
    "2016": 351,
    "2017": 391
}

In [123]:
# Filtrar os microdados para SG_AREA 'CN', TX_COR 'AZUL' e CO_PROVA 'X'
# ATENÇÃO: O Código da Prova para a Aplicação Regular muda a cada ano
df_microdados_filtrado = df_microdados[
    (df_microdados["SG_AREA"] == "CN")  # Ciência das Naturezas
    & (df_microdados["TX_COR"].str.upper() == "AZUL")  # Apenas caderno azul
    & (df_microdados["CO_PROVA"] == co_prova[year])  # Aplicação Regular
]

# Selecionar apenas as colunas de interesse dos microdados
df_microdados_sel = df_microdados_filtrado[
    ["CO_POSICAO", "NU_PARAM_B", "TX_GABARITO"]
].copy()

if year == "2016":
    df_microdados_sel["CO_POSICAO"] = df_microdados_sel["CO_POSICAO"] - 45
elif year == "2017":
    df_microdados_sel["CO_POSICAO"] = df_microdados_sel["CO_POSICAO"] + 90

In [124]:
# Certificar que os tipos das chaves de junção são compatíveis
df_questions["numero_questao"] = df_questions["numero_questao"].astype(str)
df_microdados_sel["CO_POSICAO"] = df_microdados_sel["CO_POSICAO"].astype(str)

# Fazer o cruzamento usando o número da questão (numero_questao e CO_POSICAO)
df_merged = pd.merge(
    df_questions,
    df_microdados_sel,
    left_on="numero_questao",
    right_on="CO_POSICAO",
    how="left",
)

In [125]:
# Remover a coluna CO_POSICAO (redundante)
df_merged.drop(columns=["CO_POSICAO"], inplace=True)

# Renomeando colunas
df_merged.rename(columns={
    "NU_PARAM_B": "nu_param_B",
    "TX_GABARITO": "gabarito"
}, inplace=True)


# Salvando DF resultante
df_merged.to_csv(output_path, index=False, encoding="utf-8")

print("Merge concluído.")

Merge concluído.


In [126]:
df = pd.read_csv(output_path)
df.head()

,numero_questao,enunciado,alternativas,nu_param_B,gabarito
0,46,"Em uma flauta, as notas musicais possuem \nfre...",A: L; B: L; C: L; D: L; E: 6L,2.28250,C
1,47,"Durante a aula, um professor apresentou uma \n...",A: obesidade.; B: osteoporose.; C: diabetes ti...,0.11367,D
2,48,"O cladograma representa, de forma simplificada...",A: 1.; B: 2.; C: 3.; D: 4.; E: 5.,0.56500,C
3,49,Um eletricista projeta um circuito com três lâ...,A: 2\n X.; B: X.; C: 3\n X.; D: 2X.; E: 3X.,2.22439,C
4,50,O urânio é um elemento cujos átomos contêm 92 ...,"A: 2,10; B: 7,70; C: 9,00; D: 33,0; E: 300",1.56282,D
